In [2]:
import subprocess
import os
import time
import csv

TIMEOUT = 120
REPETITIONS = 5
ENUMERATION_LIMIT = 1000

In [3]:
# Runs trapmvn with either unitary or general update scheme,
# reporting the runtime, number of fails, and the number of results.
def bench_trapmvn(model, semantics = "general"):
    cmd_trapmvn = ["python3", "-m", "trapmvn", "-c", "max", "-s", semantics, "-m", str(ENUMERATION_LIMIT), model]
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            result = subprocess.run(cmd_trapmvn, capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start
            result_count = len(result.stdout.splitlines()) - 1
            if result.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count)

In [4]:
# For maximal trap spaces, we can only test trapmvn and nothing else.
# But we can still test general/unitary semantics.
with open('max-trap-benchmark.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Model"]
    header += ["trapmvn (general)", "trapmvn (unitary)"]
    writer.writerow(header)
    print(header)
    
    for model in sorted(os.listdir("models")):
        if not model.endswith(".sbml"):
            continue
        
        row = [model]
        model = f"models/{model}"
        # Fixed points are shared across semantics.
        trapmvn_general = bench_trapmvn(model, semantics="general")
        trapmvn_unitary = bench_trapmvn(model, semantics="unitary")        
        row += [str(trapmvn_general), str(trapmvn_unitary)]
        print(row)
        
        writer.writerow(row)
        csvfile.flush()

['Model', 'trapmvn (general)', 'trapmvn (unitary)']
['146_BUDDING-YEAST-FAURE-2009.sbml', '(1.4433969443663954, 0, 12)', '(1.246937102265656, 0, 10)']
['148_AGS-cell-fate-decision.sbml', '(0.11210814081132411, 0, 11)', '(0.10999473575502634, 0, 11)']
['151_TCR-REDOX-METABOLISM.sbml', '(0.30786582324653866, 0, 6)', '(0.2564804984256625, 0, 6)']
['155_CONTROL-OF-TH1-TH2-TH17-TREG-DIFFERENTATION.sbml', '(0.11187891066074371, 0, 42)', '(0.10835150461643935, 0, 42)']
['157_CONTROL-OF-TH-DIFFERENTATION.sbml', '(0.13460688143968583, 0, 63)', '(0.14557357970625162, 0, 63)']
['159_BUDDING-YEAST-CORE.sbml', '(0.5875140024349094, 0, 10)', '(0.5878027880564332, 0, 8)']
['160_IL17-DIFFERENTIAL-EXPRESSION.sbml', '(0.1391521215438843, 0, 30)', '(0.13603760544210672, 0, 30)']
['161_DIFFERENTIATION-OF-MONOCYTES.sbml', '(0.11826782543212175, 0, 4)', '(0.11869635600596666, 0, 4)']
['167_DROSOPHILA-MESODERM.sbml', '(0.11922557689249516, 0, 29)', '(0.10552985426038504, 0, 29)']
['175_SEA-URCHIN.sbml', '(0.